In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# Configuracion de carpetas
carpeta_drive_base = "drive/My Drive/Colab Notebooks/"
carpeta_drive_prueba_red = "drive/My Drive/Colab Notebooks/Prueba red/"
carpeta_drive_cifar = "drive/My Drive/Colab Notebooks/Prueba red/cifar_100_python/"

In [0]:
import os
import sys
import tensorflow 
from keras.datasets import cifar10, mnist
from keras.applications.vgg19 import VGG19
from keras.applications.resnet_v2 import ResNet50V2
from keras.applications.mobilenet_v2 import MobileNetV2

from keras.layers import Dense, Dropout, Flatten, Input, concatenate, Conv2D, MaxPooling2D
from keras.models import Model
import cv2
import numpy as np
import importlib.util
from keras.utils import to_categorical

In [0]:
from keras.callbacks import ReduceLROnPlateau
from keras.layers import (Input, Conv2D, BatchNormalization, ZeroPadding2D,
                          GlobalAveragePooling2D, Activation, Dense, add)
from keras.models import Model
from keras import optimizers

DEPTH = 164 # or 1001
MODEL_NAME = f'ResNet{DEPTH}' # This should be modified when the model name changes.

class ResNet164():
    '''
    1. ZeroPadding2D (2, 2)
    2. 3X3 Conv2D 16
    3. ResidualBlock X 18 + 1
    4. ResidualBlock X 18 + 1
    5. ResidualBlock X 18 + 1
    6. BN + Relu
    7. GlobalAveragePooling2D
    8. FC 10 + Softmax
    '_build()' is only modified when the model changes.
    HowToUse:
        model = ResNet164()
        * all funtionalities are written in BaseModel.py
    '''
    def __init__(self):
        callbacks = [ReduceLROnPlateau(monitor = 'val_loss', factor = 0.1,
                                       patience = 10, verbose = 1)]
        optimizer = optimizers.SGD(lr=0.1, momentum=0.9, decay=1e-04)

    def _build(self):
        '''
        Builds ResNet164.
        - Deep Residual Learning for Image Recognition (https://arxiv.org/abs/1512.03385)
          => Bottleneck
          => Projection shortcut (B)
        - Identity Mappings in Deep Residual Networks (https://arxiv.org/abs/1603.05027)
          => Full pre-activation
        - Author's Implementation
          => https://github.com/KaimingHe/resnet-1k-layers/blob/master/resnet-pre-act.lua
        Returns:
            ResNet164 model
        '''
        n = (DEPTH - 2) // 9
        nStages = [16, 64, 128, 256]

        x = Input(shape = (28, 28, 1))
        y = ZeroPadding2D(padding = (2, 2))(x) # matching the image size of CIFAR-10

        y = Conv2D(nStages[0], (3, 3), padding = 'same')(y)
        y = self._layer(y, nStages[1], n, (1, 1)) # spatial size: 32 x 32
        y = self._layer(y, nStages[2], n, (2, 2)) # spatial size: 16 x 16
        y = self._layer(y, nStages[3], n, (2, 2)) # spatial size: 8 x 8
        # x = Dense(512, activation="relu")(x)
        y = BatchNormalization()(y)
        y = Activation('relu')(y)
        y = GlobalAveragePooling2D()(y)
        input2 = Input(shape=(1,1,128))

        new_dens = Dense(256)(input2)
        flatten2 = Flatten()(new_dens)
        combined = concatenate([y, flatten2])
        pre = Dense(512)(combined)

        dense = Dense(units=10, kernel_initializer="he_normal",
                      activation="softmax")(pre)

        model = Model(inputs=[x, input2], outputs=dense)
        

        return model

    def _layer(self, x, output_channel, count, strides):
        '''
        Creates a layer which consists of residual blocks as many as 'count'.
        Returns:
            A layer which consists of multiple residual blocks
        '''
        y = self._residual_block(x, output_channel, True, strides)

        for _ in range(1, count):
            y = self._residual_block(y, output_channel, False, (1, 1))

        return y

    def _residual_block(self, x, output_channel, downsampling, strides):
        '''
        Residual Block: x_{l+1} = x_{l} + F(x_{l}, W_{l})
        Returns:
            a single residual block
        '''
        bottleneck_channel = output_channel // 4

        if downsampling:
            # common BN, ReLU
            x = BatchNormalization()(x)
            x = Activation('relu')(x)

            fx = Conv2D(bottleneck_channel, (1, 1), padding = 'same', strides = strides,
                        kernel_initializer = 'he_normal')(x)
        else:
            # conv1x1
            fx = BatchNormalization()(x)
            fx = Activation('relu')(fx)
            fx = Conv2D(bottleneck_channel, (1, 1), padding = 'same',
                        kernel_initializer = 'he_normal')(fx)

        # conv3x3
        fx = BatchNormalization()(fx)
        fx = Activation('relu')(fx)
        fx = Conv2D(bottleneck_channel, (3, 3), padding = 'same',
                    kernel_initializer = 'he_normal')(fx)

        # conv1x1
        fx = BatchNormalization()(fx)
        fx = Activation('relu')(fx)
        fx = Conv2D(output_channel, (1, 1), padding = 'same',
                    kernel_initializer = 'he_normal')(fx)

        if downsampling:
            # Projection shorcut
            x = Conv2D(output_channel, (1, 1), padding = 'same', strides = strides,
                        kernel_initializer = 'he_normal')(x)

        return add([x, fx])

model = ResNet164()

In [0]:
model = model._build()

In [0]:
model.summary()

In [0]:
model = MobileNetV2(weights=None,input_shape=(28,28,3),include_top=True)
# model = Model(inputs=base_model.input, outputs=base_model.get_layer('block5_conv4 ').output)
# base_model.summary()
# x = base_model.get_layer('block5_pool').output
# x = Dense(512, activation="relu")(x)
# input2 = Input(shape=(1,1,128))

# new_dens = Dense(512)(input2)
# # flatten2 = Flatten()(new_dens)
# combined = concatenate([x, new_dens])
# pre = Dense(512)(combined)

# dense = Dense(units=10, kernel_initializer="he_normal",
#               activation="softmax")(pre)

# model = Model(inputs=[base_model.input, input2], outputs=dense)

In [40]:
#Cargando modelo de rotacion
from keras.models import model_from_json

json_file = open(carpeta_drive_prueba_red + 'model_rotation_mnist.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model_rotation = model_from_json(loaded_model_json)
# load weights into new model
model_rotation.load_weights(carpeta_drive_prueba_red +"model_rotation_mnist.h5")
print("modelo cargado")

modelo cargado


In [0]:

from keras.datasets import cifar10, mnist
# (x_train, y_train), (x_test, y_test) = cifar10.load_data()
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [46]:
from keras import backend as K
                                        
inp = model_rotation.input                                           
outputs = [layer.output for layer in model.layers[5:]]         
functors = [K.function([inp, K.learning_phase()], [out]) for out in outputs] 

layer_outs = [[func([x_train[i].reshape(1,28,28,1), 1.]) for func in functors] for i in range(len(x_train))]
features = [layer_outs[i][1][0][0] for i in range(len(layer_outs))]

InvalidArgumentError: ignored

In [0]:
features_rotation = [layer_outs[i][0][0][0] for i in range(len(layer_outs))]

In [31]:
model.compile(optimizer='adam', loss='categorical_crossentropy')

In [0]:
trainY_array = np.array(y_train)
y_train = to_categorical(trainY_array,num_classes=None, dtype='float32')

In [0]:
features_rotation= np.array(features_rotation)
features_rotation= features_rotation.reshape(x_train.shape[0],1,1,128)
x_train = x_train[:]
y_train = y_train[:]

In [0]:
y_train = y_train.reshape(y_train.shape[0],1,1,10)

In [0]:
model.fit([x_train,features_rotation],y_train,epochs=10,batch_size=1000,validation_data=[[x_train,features_rotation],y_train])


Train on 50000 samples, validate on 50000 samples
Epoch 1/10
50000/50000 [==============================] - 27s 548us/step - loss: 0.6030 - val_loss: 0.4562
Epoch 2/10
50000/50000 [==============================] - 27s 537us/step - loss: 0.4736 - val_loss: 0.3854
Epoch 3/10
50000/50000 [==============================] - 27s 533us/step - loss: 0.4027 - val_loss: 0.3359
Epoch 4/10
50000/50000 [==============================] - 27s 539us/step - loss: 0.3439 - val_loss: 0.2943
Epoch 5/10
50000/50000 [==============================] - 27s 537us/step - loss: 0.3097 - val_loss: 0.2386
Epoch 6/10
50000/50000 [==============================] - 27s 536us/step - loss: 0.2693 - val_loss: 0.2082
Epoch 7/10
50000/50000 [==============================] - 27s 536us/step - loss: 0.2473 - val_loss: 0.1988
Epoch 8/10
50000/50000 [==============================] - 27s 536us/step - loss: 0.2236 - val_loss: 0.1614
Epoch 9/10
50000/50000 [==============================] - 27s 536us/step - loss: 0.1760 - val_

In [0]:
result = model.predict([x_train,features_rotation])

In [0]:
result[0][0][0]

array([0., 0., 1., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)

In [0]:
cont = 0
for i in range(len(result)):
  if (np.argmax(y_train[i][0][0]) == np.argmax(result[i][0][0])):
    cont+=1
print(cont/len(result))


0.96192


In [0]:
# serialize model to JSON
model_json = model.to_json()
with open(carpeta_drive_prueba_red + "model_conjunto_rotacion_cifar.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights(carpeta_drive_prueba_red + "model_conjunto_rotacion_cifar.h5")
print("Saved model to disk")

Saved model to disk
